In [1]:
import pandas as pd
import numpy as np

In [2]:
search = pd.read_csv('/data/log-data-2024/search_log_09_11.csv')
search

,device_id,country,product,language,params,api,result_domain,log_time
0,2nlerTrWEoF2C1auts0L07ysLzAkOdujlbVMNy4FJXQ=,US,webOSTV 23,en-US,start_index=1&query=2022 FCS Football Quarterf...,/rest/sdp/v14.0/search/retrieval,web,2024-09-01 00:10:15.751
1,Sj5G2u9OYhVIKX+EdbJsdCZmR1j13O7rbDb0xa6XPqY=,US,webOSTV 23,en-US,ignore_hint=Y&query=JCN&check_exact_matched_cp...,/rest/sdp/v14.0/search/retrieval,"youtube,web",2024-09-01 00:10:38.382
2,Bm1RAMyJKuKLKeRZJ46vxkheUPDX6RVVLsPYpQkEqyc=,US,webOSTV 23,es-US,ignore_hint=Y&query=El bebecito rizado&check_e...,/rest/sdp/v14.0/search/retrieval,"youtube,movie,web",2024-09-01 00:10:41.311
3,2rkfHfto9ife9RMxoPMVEYjZOHqunpGe/LlIBNtxT/U=,US,webOSTV 23,en-US,query=Some like it hot&check_exact_matched_cp=...,/rest/sdp/v14.0/search/retrieval,"epg,movie,tvshow,youtube,check_exact_match,net...",2024-09-01 00:10:42.161
4,6ag8UPj/jDqSTtcxQi/eJvfXM0ABM5n1BKZqO91VX0Y=,US,webOSTV 23,en-US,preference_sports_id=&start_index=1&query=Para...,/rest/sdp/v14.0/search/retrieval,"app,movie,tvshow,youtube,web",2024-09-01 00:10:43.019
...,...,...,...,...,...,...,...,...
16102022,GE7EUlzrHA63C0UcOtZG6tJ0c8K3REb9mlJZVnt7CKA=,US,webOSTV 23,en-US,result_filter=full_match&query=search top rate...,/rest/sdp/v14.0/search/retrieval,"movie,tvshow,youtube,check_exact_match,netflix...",2024-10-31 21:01:25.875
16102023,HThXFxjqs20ssBM9rl3RrgGw1l3Gl236jwok26gVIVE=,US,webOSTV 23,en-US,item_index=2&category_id=epg&service=com.webos...,/rest/sdp/v14.0/search/logging,NaN,2024-10-31 21:01:49.249
16102024,GoMIHWLGAHSJALD3oqb3DvBV+sfuVLN1tfTHjhmTeEY=,US,webOSTV 23,es-US,result_filter=full_match&query=Niña de 13 años...,/rest/sdp/v14.0/search/retrieval,"youtube,web",2024-10-31 21:02:10.825
16102025,Q0wacxSh9w2pc/I9s6mfjCQPH3uMP7RVX/ZzpZDobWo=,US,webOSTV 23,en-US,result_filter=full_match&query=Talk slower&inp...,/rest/sdp/v14.0/search/retrieval,"youtube,web",2024-10-31 21:02:25.730


In [3]:
search['log_time'] = pd.to_datetime(search['log_time'])

In [7]:
# api 컬럼에서 logging인 데이터만 추출
search['api_logging'] = search['api'].apply(lambda x : x.split('/')[-1])
search = search[search['api_logging'] == 'logging'].reset_index(drop=True)

In [6]:
# domain이 tvshow, movie인 데이터만 추출
search['domain'] = search['params'].apply(lambda x: next((part.split('=')[1] for part in x.split('&') if part.startswith('domain=')), None))
search.head(2)

,device_id,country,product,language,params,api,result_domain,log_time,domain
0,2nlerTrWEoF2C1auts0L07ysLzAkOdujlbVMNy4FJXQ=,US,webOSTV 23,en-US,start_index=1&query=2022 FCS Football Quarterf...,/rest/sdp/v14.0/search/retrieval,web,2024-09-01 00:10:15.751,"epg,tvshow,movie"
1,Sj5G2u9OYhVIKX+EdbJsdCZmR1j13O7rbDb0xa6XPqY=,US,webOSTV 23,en-US,ignore_hint=Y&query=JCN&check_exact_matched_cp...,/rest/sdp/v14.0/search/retrieval,"youtube,web",2024-09-01 00:10:38.382,"epg,tvshow,movie,app,youtube,fitness,game,spor..."


In [10]:
search = search[search['domain'].isin(['movie', 'tvshow'])].reset_index(drop=True)

In [11]:
# params에서 content_id, content_set_id 추출
search['content_id'] = search['params'].apply(lambda x: next((part.split('=')[1] for part in x.split('&') if part.startswith('content_id=')), None))
search['content_set_id'] = search['params'].apply(lambda x: next((part.split('=')[1] for part in x.split('&') if part.startswith('content_set_id=')), None))

In [17]:
search = search[['device_id', 'log_time', 'content_id', 'content_set_id', 'api_logging']]

In [18]:
search.to_csv('./data/search_log_241127.csv', index=False)

In [20]:
search.tail()

,device_id,log_time,content_id,content_set_id,api_logging
1638667,dL6YmTivD+0GNosxil2AR2QiM+N2CW6TRNYplgjSewI=,2024-10-31 21:53:55.164,amzn1.dv.gti.26aca0c8-2438-4319-a36f-b3497004019b,com.lge.meta.crawler.amazon.AmazonCrawler_US,logging
1638668,TIO2Q8zmVduxq1lk90m/1pt3o2QKGp5L5X85LeMgOOw=,2024-10-31 21:58:15.194,content_id%3D667315%26utm_source%3Dsearch%26ut...,com.lge.meta.crawler.tubi.TubiCrawler_US,logging
1638669,NFpTclZYPrzj7b0JSPTlM0+khp7p3anJxifz2tl1oDU=,2024-10-31 21:58:31.211,TMS3499,com.lge.meta.crawler.tms.TmsCrawler,logging
1638670,WFpqNihBiCXqYgICSw//gwsJ7H1RG/9llpsjfgPZRHs=,2024-10-31 21:59:34.253,TMS17290385,com.lge.meta.crawler.tms.TmsCrawler,logging
1638671,wzEqRYvyesZFKHVdzsWpViezk9NUfSaFSerK5TAdYLU=,2024-10-31 21:00:00.294,TMS18975942,com.lge.meta.crawler.tms.TmsCrawler,logging
